In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Step 1: Load and preprocess MNIST data
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype("float32")
x_train = (x_train - 127.5) / 127.5  # scale to [-1, 1]

# Step 2: Build Generator
# Input: random noise → Output: fake image
def make_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation="relu", input_dim=100))
    model.add(layers.Dense(784, activation="tanh"))  # 28*28 pixels
    return model

# Step 3: Build Discriminator
# Input: image → Output: real (1) or fake (0)
def make_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation="relu", input_dim=784))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

# Step 4: Combine into GAN (Generator + Discriminator)
generator = make_generator()
discriminator = make_discriminator()

discriminator.trainable = False  # Freeze while training GAN
gan_input = tf.keras.Input(shape=(100,))
fake_image = generator(gan_input)
gan_output = discriminator(fake_image)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss="binary_crossentropy", optimizer="adam")

# Step 5: Training Loop
epochs = 1000
batch_size = 128

for epoch in range(epochs):
    # --- Train Discriminator ---
    # Get real images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]
    
    # Generate fake images
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_imgs = generator.predict(noise, verbose=0)
    
    # Train on real and fake
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 1)))
    
    # --- Train Generator ---
    noise = np.random.normal(0, 1, (batch_size, 100))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    
    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch {epoch} | D Loss: {d_loss_real[0]+d_loss_fake[0]:.4f} | G Loss: {g_loss:.4f}")

Epoch 0 | D Loss: 1.2977 | G Loss: 0.7529
Epoch 100 | D Loss: 10.0443 | G Loss: 0.0131
Epoch 200 | D Loss: 10.6360 | G Loss: 0.0066
Epoch 300 | D Loss: 10.8902 | G Loss: 0.0044
Epoch 400 | D Loss: 11.0668 | G Loss: 0.0033
Epoch 500 | D Loss: 11.2046 | G Loss: 0.0027
Epoch 600 | D Loss: 11.3290 | G Loss: 0.0022
Epoch 700 | D Loss: 11.4417 | G Loss: 0.0019
Epoch 800 | D Loss: 11.5431 | G Loss: 0.0017
Epoch 900 | D Loss: 11.6392 | G Loss: 0.0015
